# Data sanitization
1. Some files have more boxes than texts
2. Orientated boxes need to be removed

In [153]:
DATA_PATH='/home/sravya/data/muse/'
SYNTH_HOME = DATA_PATH+'SynthText/SynthText/'
PROCESSED = SYNTH_HOME + 'processed/'
NON_ORIENTED = SYNTH_HOME + 'non_oriented/'
import synth_utils
import importlib
importlib.reload(synth_utils)

<module 'synth_utils' from '/home/sravya/git/Muse_Deliverable/synth_utils.py'>

## Remove files that have more boxes than texts

In [ ]:
# Remove files that have more boxes than texts and save new imnames,texts and boxes as numpy arrays to disk at ./processed
save_clean_inputs(SYNTH_HOME)    
#TODO: Clean up this utility

## Remove oriented boxes

In [17]:
import numpy as np
wordBB = np.load(SYNTH_HOME + 'processed/wordBB.npy')
imnames = np.load(SYNTH_HOME + 'processed/imnames.npy')
text = np.load(SYNTH_HOME + 'processed/text.npy')

good_boxes = 0
bad_boxes = 0
file_boxes = []
file_texts = []
files = []

for index, file in enumerate(wordBB):
    boxes = []
    texts = []
    for bb in list(range(file.shape[-1])):

        xmin = file[0][0][bb]
        xmax = file[0][2][bb]
        ymin = file[1][0][bb]
        ymax = file[1][2][bb]
        if xmin<xmax and  ymin>ymax:
            good_boxes += 1
            boxes.append((xmin,ymin,xmax,ymax))
            texts.append(text[index][bb])
            box_indices.append(bb)            
        else:
            bad_boxes += 1
    if(len(boxes) != 0):
        files.append(imnames[index])
        file_boxes.append(boxes)
        file_texts.append(boxes)
#Save new set of imnames, boxes and texts
synth_utils.numpysave(NON_ORIENTED, files, file_boxes, file_texts)

# Explore and sanity checks

In [23]:
len(file_boxes), len(file_texts), len(files)

(83627, 83627, 83627)

In [24]:
good_boxes, bad_boxes

(162030, 1573031)

In [25]:
num_boxes = 0
for boxes in file_boxes:
    num_boxes += len(boxes)

In [26]:
num_boxes

162030

In [27]:
num_boxes = 0
for boxes in file_texts:
    num_boxes += len(boxes)

In [28]:
num_boxes

162030

# CreateTF records

In [154]:
# import numpy as np
files = np.load(NON_ORIENTED + 'imnames.npy')
file_boxes = np.load(NON_ORIENTED + 'wordBB.npy')
file_texts = np.load(NON_ORIENTED + 'text.npy')

In [155]:
import sys
sys.path.append("/home/sravya/git/models")
from object_detection.utils import dataset_util

In [156]:
import numpy as np
import utils
from PIL import Image
PATH=DATA_PATH+'SynthText/SynthText_split/'
def create_tf_record(filename, wordBB, text):
    """ returns TFRecord for each file"""
    img = Image.open(PATH + filename)
    width, height = img.size
    #Add zero check
    with tf.gfile.GFile((PATH + filename), 'rb') as fid:
        encoded_image_data = fid.read()
    filename = filename.encode('utf8')

    image_format = b'jpg'
    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = []
        
    for index, bb in enumerate(wordBB):
        #print(bb[0],bb[1],bb[2],bb[3])
        x_min, y_min, x_max, y_max = bb[0]/width, bb[1]/height,bb[2]/width, bb[3]/height
        #Make sure bb lies within the image
        #print(x_min,y_min,x_max,y_max)
        if(x_min < 1 and y_min <1 and x_max <1 and y_max <1):
            xmins.append(x_min)
            ymins.append(y_min)
            xmaxs.append(x_max)
            ymaxs.append(y_max)
            classes_text.append('Text'.encode('utf8'))
            classes.append(1)
    #print(xmins,xmaxs,ymins,ymaxs)
#     #Sanity check none of the values are Nans
#     if not np.all(np.isfinite(xmins)):
#         print(xmins)
#     if not np.all(np.isfinite(ymins)):
#         print(ymins)
#     if not np.all(np.isfinite(xmaxs)):
#         print(ymaxs)
        
    if (len(xmins) != 0):
        tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
    else:
        #print('Skipped filename: ' + str(filename))
        return None
    return tf_example

In [161]:
len(files)

83627

In [182]:
import random
indices = list(range(0,len(files)))
len(indices)

83627

In [184]:
random.shuffle(indices)

In [185]:
indices[:2]

[19984, 67635]

In [175]:
import tensorflow as tf
from tqdm import tqdm
skipped = 0
writer = tf.python_io.TFRecordWriter('data/training.record')
for index in tqdm(indices[:58538], total=58538):
    tf_record = create_tf_record(files[index][0], file_boxes[index], file_texts[index])
    if tf_record is not None:
        writer.write(tf_record.SerializeToString())
    else:
        skipped += 1
writer.close()

100%|██████████| 58538/58538 [00:20<00:00, 2854.35it/s]


In [176]:
skipped

906

In [178]:
writer = tf.python_io.TFRecordWriter('data/valid.record')
for index in tqdm(indices[58538:75264], total=16726):
    tf_record = create_tf_record(files[index][0], file_boxes[index], file_texts[index])
    if tf_record is not None:
        writer.write(tf_record.SerializeToString())
    else:
        skipped += 1
writer.close()

100%|██████████| 16726/16726 [00:05<00:00, 3108.78it/s]


In [179]:
skipped

1185

In [180]:
writer = tf.python_io.TFRecordWriter('data/test.record')
for index in tqdm(indices[75264:83627], total=8363):
    tf_record = create_tf_record(files[index][0], file_boxes[index], file_texts[index])
    if tf_record is not None:
        writer.write(tf_record.SerializeToString())
    else:
        skipped += 1
writer.close()

100%|██████████| 8363/8363 [00:02<00:00, 3092.44it/s]


In [181]:
skipped

1307

In [67]:
#Move all files into split folder
from random import shuffle
from glob import glob
import os

sample_files = glob(DATA_PATH+'SynthText/SynthText_sample/*/*')
len(sample_files)

for path in sample_files:
    newfile = path.replace('sample','split')
    os.renames(path,newfile)
    print(path,newfile)

In [1]:
import matplotlib.pyplot as plt

In [ ]:
plt.text()